# Welcome to Test! 
---

## How does it work?

This welcome notebook provides two main features:

### 1. Browse and Open Notebooks

You will find below an interactive table with all available notebooks, each designed to facilitate a specific aspect of experiment management. Simply click on any "Open the notebook" button to launch it in JupyterLab.

### 2. Check for Updates

Stay up-to-date with the latest notebook versions! Enter your GitHub Personal Access Token and click "Test Connection" to compare your local notebooks with the latest versions from the GitHub repository. You can update any outdated notebooks directly with a single click.


In [ ]:
# Run this cell to see the interactive notebook list ⬇️⬇️⬇️

from IPython.display import display, Markdown, HTML, clear_output
from ipywidgets import widgets, GridspecLayout
from pathlib import Path
import requests
import base64
import yaml

# Define notebooks with their metadata
notebooks = DICT_OF_NOTEBOOKS

# Load the local information from all the notebooks
local_latest_version_path = Path("..") / "notebook_latest_versions.yaml"
local_latest_versions = yaml.safe_load(local_latest_version_path.read_text())

# Load the local construct information
construct_file_path = Path("..") / "construct.yaml"
construct_info = yaml.safe_load(construct_file_path.read_text())
local_project_version = construct_info.get('version', '0.0.0')

# Load the online information (GitHub) from all the notebooks
github_owner = "GITHUB_OWNER"
github_repo_name = "GITHUB_REPO_NAME"
github_branch = "main"  # Change to "dev" if you want to check against dev branch

def load_table(version_response, project_version_response, notebooks):
    if version_response is None:
        display(Markdown("⚠️ Not checking versions. Could not access the version file from GitHub."))
        online_latest_versions = local_latest_versions
    else:
        try:
            online_latest_versions = yaml.safe_load(
                base64.b64decode(version_response.json()['content']).decode('utf-8')
            )
            if not isinstance(online_latest_versions, dict):
                display(Markdown("⚠️ Not checking versions. The online version file format is invalid."))
                online_latest_versions = local_latest_versions
        except Exception as e:
            display(Markdown(f"⚠️ Not checking versions. Could not parse version file: {e}"))
            online_latest_versions = local_latest_versions

    if project_version_response is None:
        online_project_version = local_project_version
    else:
        try:
            project_construct_info = yaml.safe_load(
                base64.b64decode(project_version_response.json()['content']).decode('utf-8')
            )
            online_project_version = project_construct_info.get('version', '0.0.0')
        except Exception as e:
            display(Markdown(f"⚠️ Could not parse construct.yaml file: {e}"))
            online_project_version = local_project_version

    if online_project_version != local_project_version:
        display(Markdown(f"❌ The project version is outdated. Local version: {local_project_version}, Online version: {online_project_version}"))
        display(Markdown("Please update the project to the latest version by following the update instructions. Until you update the project, you will not be able to update individual notebooks."))
        project_needs_update = True
    else:
        display(Markdown(f"✅ The project version is up-to-date. Version: {local_project_version}"))
        project_needs_update = False

    # Go through all the notebooks and compare versions
    num_rows = len(notebooks)
    grid = GridspecLayout(1 + num_rows, 7)
    grip_output = widgets.HTML()

    # Add custom CSS for styling
    display(widgets.HTML("""
    <style>
    .grid-header {
        background-color: #ff6600 !important;
        color: white !important;
        font-weight: 600 !important;
        padding: 12px !important;
        text-align: center !important;
    }
    .centered-cell {
        text-align: center !important;
        display: flex !important;
        align-items: center !important;
        justify-content: center !important;
    }
    </style>
    """))


    grid[0, 0] = widgets.HTML("<div class='grid-header'><b>Notebook Name</b></div>")
    grid[0, 1] = widgets.HTML("<div class='grid-header'><b>Notebook Topic</b></div>")
    grid[0, 2] = widgets.HTML("<div class='grid-header'><b>Description</b></div>")
    grid[0, 3] = widgets.HTML("<div class='grid-header'><b>Local Version</b></div>")
    grid[0, 4] = widgets.HTML("<div class='grid-header'><b>Status</b></div>")
    grid[0, 5] = widgets.HTML("<div class='grid-header'><b>Update?</b></div>")
    grid[0, 6] = widgets.HTML("<div class='grid-header'><b>⬇️ Click to Open the Notebook ⬇️</b></div>")

    needs_update = False

    idx = 1
    nb = {}
    
    for main_folder in online_latest_versions:
        if main_folder in local_latest_versions:
            for subfolder in online_latest_versions[main_folder]:
                # Find the notebook path from the notebooks list
                notebook_found = False
                for n in notebooks:
                    if subfolder in n['path']:
                        nb = n
                        notebook_found = True
                        break

                if not notebook_found:
                    continue
                    
                grid[idx, 0] = widgets.HTML(f"<div style='text-align: center;'>{nb['name']}</div>")
                grid[idx, 1] = widgets.HTML(f"<div style='text-align: center;'>{main_folder}</div>")
                        
                if nb:
                    grid[idx, 2] = widgets.HTML(f"<div style='text-align: center;'>{nb['description']}</div>")
                else:
                    grid[idx, 2] = widgets.HTML("<div style='text-align: center;'>-</div>")

                if subfolder in local_latest_versions.get(main_folder, {}):
                    online_version = online_latest_versions[main_folder][subfolder]
                    local_version = local_latest_versions[main_folder][subfolder]
                    
                    grid[idx, 3] = widgets.HTML(f"<div style='text-align: center;'>{local_version}</div>")

                    if project_needs_update:
                        grid[idx, 4] = widgets.HTML("<div style='text-align: center;'><span style='color: red;'>❌ Project Outdated</span></div>")
                        grid[idx, 5] = widgets.HTML("<div style='text-align: center;'>-</div>")
                    elif online_version == local_version:
                        grid[idx, 4] = widgets.HTML("<div style='text-align: center;'>✅ Up-to-date</div>")
                        grid[idx, 5] = widgets.HTML("<div style='text-align: center;'>-</div>")
                    else:
                        grid[idx, 4] = widgets.HTML(f"<div style='text-align: center;'><span style='color: red;'>❌ Needs Update to {online_version}</span></div>")
                        
                        def button_update(main_folder, subfolder, row_idx):
                            def show(button):
                                button.disabled = True
                                button.description = "Updating..."
                                
                                # Download the notebook
                                notebook_url = f"https://api.github.com/repos/{github_owner}/{github_repo_name}/contents/notebooks/{main_folder}/{subfolder}/{subfolder}.ipynb?ref={github_branch}"
                                if private_repository:
                                    if github_token.value:
                                        notebook_response = requests.get(notebook_url, headers={"Authorization": f"token {github_token.value}"})
                                    else:
                                        grip_output.value = "❌ Please check your Presonal Access Token"
                                else:
                                    notebook_response = requests.get(notebook_url)
                                
                                if notebook_response.status_code == 200:
                                    try:
                                        # Decode the notebook content
                                        notebook_content = base64.b64decode(
                                            notebook_response.json()['content']
                                        ).decode('utf-8')
                                        
                                        # Save to local path
                                        local_notebook_path = Path(".") / main_folder / f"{subfolder}.ipynb"
                                        local_notebook_path.parent.mkdir(parents=True, exist_ok=True)
                                        local_notebook_path.write_text(notebook_content, encoding='utf-8')
                                        
                                        # Update the local version YAML
                                        local_latest_versions[main_folder][subfolder] = online_latest_versions[main_folder][subfolder]
                                        local_latest_version_path.write_text(yaml.dump(local_latest_versions, default_flow_style=False))
                                        
                                        grip_output.value = f"✅ Updated {subfolder} to version {online_latest_versions[main_folder][subfolder]}"
                                        
                                        # Update the grid to show up-to-date
                                        grid[row_idx, 3] = widgets.HTML(f"<div style='text-align: center;'>{online_latest_versions[main_folder][subfolder]}</div>")
                                        grid[row_idx, 4] = widgets.HTML("<div style='text-align: center;'>✅ Up-to-date</div>")
                                        grid[row_idx, 5] = widgets.HTML("<div style='text-align: center;'>-</div>")
                                        
                                    except Exception as e:
                                        grip_output.value = f"❌ Error saving notebook: {e}"
                                        button.disabled = False
                                        button.description = "Update"
                                else:
                                    grip_output.value = f"❌ Failed to download notebook (HTTP {notebook_response.status_code})"
                                    display(notebook_response.text[:300])
                                    button.disabled = False
                                    button.description = "Update"
                                
                            return show

                        update_button = widgets.Button(description="Update", disable=False) 

                        update_button.on_click(button_update(main_folder, subfolder, row_idx=idx))

                        grid[idx, 5] = update_button

                        needs_update = True
                else:
                    grid[idx, 3] = widgets.HTML("<div style='text-align: center;'>Not Found</div>")
                    grid[idx, 4] = widgets.HTML("<div style='text-align: center;'>❌ Missing Notebook</div>")
                    needs_update = True

                # Open button (centered)
                open_link = widgets.HTML(
                    f'<div class="centered-cell"><a href="{nb["path"]}" target="_blank" style="display:inline-block;padding:8px 16px;background-color:#2196F3;color:white;text-decoration:none;border-radius:4px;font-weight:600;">Open the notebook</a></div>'
                )
                
                grid[idx, 6] = open_link

                idx += 1
        else:
            for subfolder in online_latest_versions[main_folder]:
                grid[idx, 0] = widgets.HTML(f"<div style='text-align: center;'>{subfolder}</div>")
                grid[idx, 1] = widgets.HTML(f"<div style='text-align: center;'>{main_folder}</div>")
                grid[idx, 2] = widgets.HTML("<div style='text-align: center;'>-</div>")
                grid[idx, 3] = widgets.HTML("<div style='text-align: center;'>Not Found</div>")
                grid[idx, 4] = widgets.HTML("<div style='text-align: center;'><span style='color: red;'>❌ Missing Notebook</span></div>")
                grid[idx, 5] = widgets.HTML("<div style='text-align: center;'>-</div>")
                grid[idx, 6] = widgets.HTML("<div style='text-align: center;'>-</div>")
                needs_update = True

                idx += 1

        
    if needs_update:
        display(Markdown("⚠️ Some notebooks need to be updated."))
    else:
        display(Markdown("✅ All notebooks are up-to-date."))

    display(grid, grip_output)

version_file_path = "notebooks/notebook_latest_versions.yaml"
version_url = f"https://api.github.com/repos/{github_owner}/{github_repo_name}/contents/{version_file_path}?ref={github_branch}"

project_version_file_path = "construct.yaml"
project_version_url = f"https://api.github.com/repos/{github_owner}/{github_repo_name}/contents/{project_version_file_path}?ref={github_branch}"
try:
    version_response = requests.get(version_url)
    project_version_response = requests.get(project_version_url)
    error = False
except Exception as e:
    load_table(None, None, notebooks)
    error = True

# Debug: show status and raw response if not 200
if not error and version_response.status_code != 200:
    private_repository = True
    explanation = widgets.HTML(
        value=f"""
        It looks like your repository might be private, please provide a Personal Access Token (PAT) to access it.
        If you don't have one, you can create it by following <a href="https://github.com/CellMigrationLab/LabConstrictor/blob/main/.tools/docs/personal_access_token.md" target="_blank" style="color: #0066cc; text-decoration: underline;">this guide</a>.
        """
    )

    github_token = widgets.Password(
        value="",
        description="*Personal Access Token:",
        placeholder="e.g. ghp_XXXXXXXXXXXXXXXXXXXX",
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='99%')
    )
    
    test_button = widgets.Button(
        description="Load Private GitHub Repo",
        button_style='success',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='99%')
    )

    output_area = widgets.HTML()
    table_area = widgets.Output()

    def on_test_button_clicked(b):
        headers = {
            "Authorization": f"token {github_token.value}"
        }
        auth_version_response = requests.get(version_url, headers=headers)    
        auth_project_version_response = requests.get(project_version_url, headers=headers)
        if auth_version_response.status_code == 200 and auth_project_version_response.status_code == 200:    
            output_area.value = "✅ Successfully accessed private repository!"
            with table_area:
                clear_output()
                load_table(auth_version_response, auth_project_version_response, notebooks)
        else:
            output_area.value = f"❌ Failed to access private repository (HTTP {auth_version_response.status_code})."
            output_area.value += f"<br><pre>{auth_version_response.text[:500]}</pre>"
            with table_area:
                clear_output()
                load_table(None, None, notebooks)
    test_button.on_click(on_test_button_clicked)

    display(explanation, github_token, test_button, output_area, table_area)
elif not error:
    private_repository = False
    load_table(version_response, project_version_response, notebooks)